In [1]:
import torch
import torch.nn as nn
from torch_geometric.nn.conv import MessagePassing

from common.graph_gen import generate_random_graph

## `MessagePassing` Base Class in PyG

The `MessagePassing` base class in PyG implements the message passing scheme as follows:

$$
x'_i=
\underbrace{
f_\theta \left(\mathbf{x}_i,
\underbrace{ 
\bigoplus_{j \in \mathcal{N}(i)} 
}_{\text{(2) aggregation}}
\underbrace{
g_\theta\left(\mathbf{x}_i, \mathbf{x}_j, \mathbf{e}_{ij}\right)
}_{\text{(1) message}}
\right)
}_{\text{(3) update}},
$$

where 
- $x_i \in \mathbb{R}^{d_n}$ and $x'_i \in \mathbb{R}^{d_n'}$ are the input and updated node features (embeddings), respectively.
- $e_{ij} \in \mathbb{R}^{d_{e}}$ is the edge feature between nodes $i$ and $j$.
- $\bigoplus$ is a differentiable and permutation-invariant function (e.g., summation, mean, maximum).
- $g_\theta : \mathbb{R}^{d_n} \times \mathbb{R}^{d_n} \times \mathbb{R}^{d_{e}} \rightarrow \mathbb{R}^{d''}$ is a trainable edge function (e.g., MLP).
- $f_\theta : \mathbb{R}^{d_n} \times \mathbb{R}^{d''} \rightarrow \mathbb{R}^{d'_n}$ is a trainable mapping function (e.g., MLP).
- $\mathcal{N}(i)$ is the set of neighbors of node $i$.


This message passing implementation is flexible and can be used to implement various GNN variants, including GCN, GAT, GraphSAGE, Interaction Layers, etc. In PyG, the `MessagePassing` triggers the message, aggregation, and update functions in the order of (1) -> (2) -> (3) while `propgate` the messages. In the following tutorial, we will demonstrate how to implement the components of message passing (message, aggregation, and update) to create different graph convolution layers.

## Very First Message Passing Layer; NaiveGCN

We will use the `MessagePassing` base class to implement the NaiveGCN layer that performs the following message passing scheme:

$$
x'_i = \sigma \left(\sum_{j \in \mathcal{N}(i)} \left(W\mathbf{x}_j+b\right) \right),
$$
where $\sigma$ is an non-linear activations (e.g. ReLU, Tanh, SiLU, GELU, ...)

In [2]:
# Yes this is it! We are done with the implementation of NaiveGCN.

class NaiveGCN(MessagePassing):
    
    def __init__(self, dim:int, act:'str'='ReLU'):
        
        super().__init__(aggr='add') # Aggregates messages with Summation (i.e., addition).        
        self.linear = nn.Linear(dim, dim)
        self.act = getattr(nn, act)()
        
    def forward(self, x, edge_index):
        x = self.linear(x) # Perform Wx+b
        x = self.propagate(x=x, edge_index=edge_index) # Propagate messages and Aggregate them with Summation
        x = self.act(x) # Apply activation function
        return x

In [3]:
num_node = 11
n_dim = 16

g = generate_random_graph(num_node=num_node, 
                          node_feat_dim=n_dim,
                          edge_feat_dim=n_dim)

In [4]:
conv = NaiveGCN(dim=n_dim)
gc_out = conv(g.x, g.edge_index)
print(gc_out.shape)

torch.Size([11, 16])


## Can we have a more minute control over messaging passing scheme?
### `message` method of `MessagePassing` class

The NaiveGCN can be implemented without explicit message generation as the messages are the source node features. However, in many cases, we often employ more sophisticated message generation schemes that generates "message" from source, destination, and edge features (if applicable). For example, the following message generation scheme is used in 
[Edge Convolution](https://arxiv.org/abs/1801.07829):

$$
x'_i = \max_{j \in \mathcal{N}(i)} h_\theta\left(x_i, x_j-x_i\right),
$$

where $h_\theta$ is an learnable function

In [5]:
class EdgeConv(MessagePassing):
    
    def __init__(self, dim:int):
        super().__init__(aggr='max')
        self.h = nn.Sequential(
            nn.Linear(dim*2, dim),
            nn.ReLU(),
            nn.Linear(dim, dim)
        )
    
    def forward(self, x, edge_index):
        return self.propagate(edge_index, x=x)
    
    def message(self, x_i, x_j):
        # By overriding this function, we can specify how messages are computed by
        
        # x_i is the source node and x_j is the target node
        # x_i, x_j has shape [E, dim], where E is the number of edges
        msg = torch.cat([x_i, x_j-x_i], dim=-1) # [E, 2*dim]
        msg = self.h(msg) # [E, dim]
        return msg

In [6]:
conv = EdgeConv(dim=n_dim)
gc_out = conv(g.x, g.edge_index)
print(gc_out.shape)

torch.Size([11, 16])


### `Update` method of `messagePassing` class

The update method is used to update the node features using the aggregated messages. As a running example, we will consider a simplified Interaction Network layer, an iconic GNN model for learning the interaction between two objects (e.g., atoms, nodes, etc.) in a graph. The update function of the Interaction Network layer is defined as follows:
$$
\begin{align}
e'_{ij} &= f_\theta(x_i, x_j, e_{ij}), \\
x'_{i} &= g_\theta(x_i, \sum_{j \in \mathcal{N}(i)} e'_{ij}),
\end{align}
$$

where $f_\theta$ and $g_\theta$ are edge and node updater, respectively. The updaters are often implemented with learnable functions (e.g., MLP)

In [7]:
class InteractionNetworkLayer(MessagePassing):
    
    def __init__(self, dim:int):
        super().__init__(aggr='add')
        self.f = nn.Sequential(
            nn.Linear(dim*3, dim), # Assuming edge are node features are of same dimension
            nn.ReLU(),
            nn.Linear(dim, dim)
        )
        
        self.g = nn.Sequential(
            nn.Linear(dim*2, dim),
            nn.ReLU(),
            nn.Linear(dim, dim)
        )

    def forward(self, x, edge_index, edge_attr):
        updated_ef = self.edge_update(x, edge_index, edge_attr)
        updated_nf = self.propagate(edge_index, x=x, edge_attr=edge_attr)    
        return updated_nf, updated_ef
    
    def edge_update(self, x, edge_index, edge_attr):
        row, col = edge_index
        x_i, x_j = x[row], x[col] # src and dst node features
        return self.f(torch.cat([x_i, x_j, edge_attr], dim=-1)) # Eq (1)

    # Eq (2) related
    def message(self, edge_attr):
        return edge_attr
    
    def update(self, aggr_msg, x): 
        # !!! The update function takes in the aggregated messages as the first argument !!!
        # The other arguments are any arguments passed to the propagate function.
        
        # Maybe good to practice
        # Try to pass additional arguments that is not passed to self.propagate such as 'y'
        # by changing 'def update(self, aggs_msg, x)' to 'def update(self, aggs_msg, x, y)'
        return self.g(torch.cat([x, aggr_msg], dim=-1))

In [8]:
conv = InteractionNetworkLayer(dim=n_dim)
updated_nf, updated_ef  = conv(g.x, g.edge_index, g.edge_attr)
print(updated_nf.shape, updated_ef.shape)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 11 but got size 8 for tensor number 1 in the list.